In [17]:

from commonfunctions import *
from Pre_Processing_Functions import *
from skimage.transform import resize
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import cv2
from sklearn.cluster import KMeans
from skimage.feature import greycomatrix
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#************************************************#
#***********************ALI*********************#
#************************************************#
#histogram equalization
def equalize(image):
    # Apply equalization to the image
    return cv2.equalizeHist(image)

def Histo_Equ_image(image):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return equalize(image)


def erosion(binary_image, window_size):
    height, width = binary_image.shape
    half_window = window_size // 2
    result_image = np.zeros_like(binary_image)

    for i in range(half_window, height - half_window):
        for j in range(half_window, width - half_window):
            window = binary_image[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1]
            result_image[i, j] = np.min(window)

    return result_image


def dilation(binary_image, window_size):
    height, width = binary_image.shape
    half_window = window_size // 2
    result_image = np.zeros_like(binary_image)

    for i in range(half_window, height - half_window):
        for j in range(half_window, width - half_window):
            window = binary_image[i - half_window:i + half_window + 1, j - half_window:j + half_window + 1]
            result_image[i, j] = np.max(window)

    return result_image

def invert_image(image):
    # Invert the image
    return 255 - image
'''
def Pre_Processing_Func(image_path):

    # load the original image
    image = cv2.imread(image_path)

    Histo_Equ_After = Histo_Equ_image(image_path)

    histo_inverted=invert_image(Histo_Equ_After)

    histo_inverted=histo_inverted/255

    binary_image=histo_inverted > 0.7

    after_erosion=erosion(binary_image,3)

    Opened_image=dilation(after_erosion,7)

    #histogram
    #hist = cv2.calcHist([Opened_image], [0], None, [256], [0, 256])

    return Opened_image
    '''
def process_image(image):
    # Perform necessary image processing operations here
    # load the original image


    Histo_Equ_After = Histo_Equ_image(image)

    histo_inverted=invert_image(Histo_Equ_After)

    histo_inverted=histo_inverted/255

    binary_image=histo_inverted > 0.7

    #after_erosion=erosion(binary_image,3)

    #Opened_image=dilation(after_erosion,7)

    hist, hist_centers = histogram(binary_image)

    #show_images([binary_image,image],['Binary_Image','before'])

    return hist



def detect_and_describe(img):

    img = np.array(img)[:, :, ::-1] # Convert PIL image to numpy array and reverse RGB channels (BGR -> RGB)

    image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert grayscale image to 8-bit unsigned integer (np.uint8) for SIFT algorithm
    # Read image in grayscale
    #image = rgb2gray(img)

    # Initialize SIFT
    sift = cv2.SIFT_create(nfeatures=200)

    # Detect keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image,None)
    #descriptors=descriptors.tolist()

    return keypoints, descriptors


def post_pre_processing(image):
    
    #resize image
    nimg = cv2.resize(image,(250,400))
    
    # Convert to grayscale
    gray = cv2.cvtColor(nimg, cv2.COLOR_BGR2GRAY)
    
    # Apply histogram equalization
    gray = cv2.equalizeHist(gray)

    #noise removal
    gray = cv2.fastNlMeansDenoising(gray, h=10, searchWindowSize=100, templateWindowSize=7)

    # Apply binary threshold
    _, binary = cv2.threshold(gray, 80, 150, cv2.THRESH_BINARY)
    return binary
   #remove noise
   # gray = cv2.bilateralFilter(gray,11,75,75)
    
    # Apply Canny edge detection with low and high thresholds
    #edges = cv2.Canny(gray, 100, 300)
    
    #return gray


def process_image(image):
    


    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary threshold
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)


    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        # Loop through each contour
    for contour in contours:
        # Approximate contour
        epsilon = 0.005 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # If contour has four points, it's a rectangle
        if len(approx) == 4:
            # Calculate aspect ratio
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = float(w)/h

            # If aspect ratio is not close to 1, it's a non-perfect rectangle
            #if aspect_ratio >2.1 and <2.5 :
            if 1.9 < aspect_ratio < 2.4 and w > 50:
                #cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)

                # Crop the image using the bounding rectangle coordinates
                cropped_image = image[y:y+h, x:x+w]
                #show_images([cropped_image,image],['cropped_image','green_border'])
                return cropped_image
            else:
                return image


In [19]:
#************************************************#
#***********************MARC*********************#
#************************************************#
def hist_hsv(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    #hist = cv2.calcHist([hsv_image], [0], None, [256], [0, 256])
    #hist /= hist.sum()

    # Get the H channel
    image1_h = hsv_image[:, :, 0]

    # Get the S channel
    image1_s = hsv_image[:, :, 1]

    # Calculate the histograms for S, H, and V channels
    hist_h = cv2.calcHist([image1_h], [0], None, [256], [0, 256])
    hist_s = cv2.calcHist([image1_s], [0], None, [256], [0, 256])
    hist_hs=cv2.calcHist([hsv_image[:,:,:1]],[0], None, [256], [0,256])


    # Normalize the histograms
    hist_h /= hist_h.sum()
    hist_s /= hist_s.sum()
    hist_hs /= hist_hs.sum()

    return hist_h,hist_s,hist_hs


def HoG(image):
    # print(img.shape)
    # Resize the image to 130x276
    resized_img = cv2.resize(image, (276, 130))
    # print(resized_img.shape)

    # Convert the original image to gray scale
    resized_img_gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Specify the parameters for our HOG descriptor
    win_size = (64, 128)  # You need to set a proper window size
    block_size = (16, 16)
    block_stride = (8, 8)
    cell_size = (8, 8)
    num_bins = 9

    # Set the parameters of the HOG descriptor using the variables defined above
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)

    # Compute the HOG Descriptor for the gray scale image
    hog_descriptor = hog.compute(resized_img_gray)
    return hog_descriptor

def Sift(train_image):
  sift = cv2.SIFT_create(nfeatures=200)
  # Convert the training image to RGB
  training_image = cv2.cvtColor(train_image, cv2.COLOR_BGR2RGB)
  # Convert the training image to gray scale
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
  train_keypoints, train_descriptor = sift.detectAndCompute(training_gray, None)

  return train_descriptor.flatten()

def extract_sift_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures=200,nOctaveLayers=3,contrastThreshold=0.01)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    return descriptors[0].flatten() if descriptors is not None else np.array([])


def get_pixel(img, center, x, y):
  new_value = 0
  try:
    if img[x][y] >= center:
      new_value = 1
  except:
    pass
  return new_value

def lbp_calculated_pixel(img, x, y):
  center = img[x][y]
  val_ar = []
  val_ar.append(get_pixel(img, center, x-1, y-1))
  val_ar.append(get_pixel(img, center, x-1, y))
  val_ar.append(get_pixel(img, center, x-1, y + 1))
  val_ar.append(get_pixel(img, center, x, y + 1))
  val_ar.append(get_pixel(img, center, x + 1, y + 1))
  val_ar.append(get_pixel(img, center, x + 1, y))
  val_ar.append(get_pixel(img, center, x + 1, y-1))
  val_ar.append(get_pixel(img, center, x, y-1))
  power_val = [1, 2, 4, 8, 16, 32, 64, 128]
  val = 0
  for i in range(len(val_ar)):
    val += val_ar[i] * power_val[i]
  return val

def LBP_matrix(img):
  nimg = cv2.resize(img,(250,400))
  height, width, _ = nimg.shape
  img_gray = cv2.cvtColor(nimg, cv2.COLOR_BGR2GRAY)
  img_lbp = np.zeros((height, width), np.uint8)
  for i in range(0, height):
    for j in range(0, width):
      img_lbp[i, j] = lbp_calculated_pixel(img_gray, i, j)
  return img_lbp[0].flatten()


def Sift_BAG(training_image):
  test_image=cv2.resize(training_image,(1208,720))
  sift = cv2.SIFT_create()
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
  train_keypoints, train_descriptor = sift.detectAndCompute(training_image, None)

  return train_keypoints, train_descriptor

def bag_of_words(descriptors, num_clusters):
    # Stack all descriptors vertically in a numpy array
    descriptors = np.vstack(descriptors)

    # Perform k-means clustering and vector quantization
    kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(descriptors)

    return kmeans

# Now, for each image, we can extract its BoW feature vector like this:
def Sift_features(kmeans, descriptors, num_clusters):
    # Predict the closest cluster each feature in descriptors belongs to.
    words = kmeans.predict(descriptors)

    # Build a histogram of clustered word counts
    word_hist = np.zeros(num_clusters, dtype=np.float32)
    for word in words:
        word_hist[word] += 1.0

    return word_hist

In [20]:
#****************************************#
#****************Co-pilot****************#
#****************************************#

#write the definition for the functions used in the Bag_of_words function
def Sift_BAG(training_image):
  test_image=cv2.resize(training_image,(1208,720))
  sift = cv2.SIFT_create(nfeatures=200)
  training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
  train_keypoints, train_descriptor = sift.detectAndCompute(training_image, None)

  return train_keypoints, train_descriptor
def bag_of_words(vocabulary, descriptor):
    # Stack all descriptors vertically in a numpy array
    descriptors = np.vstack(descriptor)

    # Perform k-means clustering and vector quantization
    kmeans = KMeans(n_clusters=vocabulary, random_state=0).fit(descriptors)

    return kmeans
def Sift_features(kmeans, descriptors, num_clusters):
    # Predict the closest cluster each feature in descriptors belongs to.
    words = kmeans.predict(descriptors)

    # Build a histogram of clustered word counts
    word_hist = np.zeros(num_clusters, dtype=np.float32)
    for word in words:
        word_hist[word] += 1.0

    return word_hist

#Write a code to implement the Bag of words concept on an image and returns a feature vector
def Bag_of_words(image):
    #Resize image
    image=cv2.resize(image,(1208,720))
    # Convert the training image to RGB
    training_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert the training image to gray scale
    training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)
    train_keypoints, train_descriptor = Sift_BAG(training_image)
    kmeans = bag_of_words(train_descriptor, 100)
    word_hist = Sift_features(kmeans, train_descriptor, 100)
    return word_hist

#write a function and its dependancies to get the bag of words for an image
def get_bow(image, k=100):
    #get the sift features for the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures=200,nOctaveLayers=3,contrastThreshold=0.01)
    keypoints, des = sift.detectAndCompute(gray, None)
    #get the clusters for the features
    kmeans = KMeans(n_clusters=k, random_state=0).fit(des)
    #get the histogram of the sift features
    hist = np.zeros(k,dtype=float)
    for d in des:
        #get the closest cluster center for each feature
        idx = kmeans.predict(d.reshape(1,-1))
        #increment the count of the cluster center
        hist[idx] += 1
    return hist

#write code for greycomatrix function used in get_glcm

def get_average_glcm(image):
    image=rgb2gray(image[:,:,:3]).astype(np.uint8)
    angles = [0,np.pi/8, np.pi/4, np.pi/2, 3*np.pi/4, 2*np.pi]  # Specify the angles for GLCM calculation
    distances = [1]  # Specify the distance for GLCM calculation
    glcm = greycomatrix(image, distances, angles, levels=256, symmetric=True, normed=True)
    average_glcm = np.mean(glcm, axis=2)  # Take the average along the third axis
    return average_glcm



In [21]:
# prepare data
input_dir = 'Data_set/'
categories = ['1EGP','1SAR', '5EGP','5SAR','10EGP','10SAR','20EGP','50EGP','50SAR','100EGP','100SAR','200EGP','500SAR']

data = []
feature_1 = []
feature_2 = []
feature_3 = []
feature_4 = []
feature_5 = []
feature_6 = []
feature_7 = []
#sifts = []
#data2=[]
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)

        img = io.imread(img_path)
        
        #pre-processing
        process_img=process_image(img)
        # Convert the image to HSV color space
        #hist_h, hist_s, hist_hs=hist_hsv(process_img)
        #new_img=post_pre_processing(process_img)
        #HOG_class=HoG(process_img)
        #BOWdz=get_bow(process_img)
        #LBP_mat=LBP_matrix(img)
        #key,descriptorz=detect_and_describe(img)
        #Ali_image = process_image(img)
        #hist=cv2.calcHist([Ali_image],[0],None,[256],[0,256])
        #descriptorz=Sift(img)
        #descriptorz=extract_sift_features(img)
        #descrop=detect_and_describe(img)
        #sifts.append(descrop)
        #feature_1.append(hist_h.flatten())
        #feature_2.append(hist_s.flatten())
        #feature_3.append(hist_hs.flatten())
        #feature_4.append(HOG_class.flatten())
        #feature_5.append(BOWdz)
        #feature_5.append(new_img.flatten())
        #feature_5.append(LBP_mat)
        #feature_5.append(descriptorz)
        #feature_6.append(Ali_image.flatten())
        #********BAG**********#
        #descriptorz=Sift_BAG(img)
        # Assuming train_descriptors is a list of descriptors for each training image
        #kmeans = bag_of_words(descriptorz, num_clusters=100)
        #feature_vector = Sift_features(kmeans, descriptorz, num_clusters=100)
        #feature_6.append(feature_vector)
        
        #******GLCM*********#
        GLCM2=get_average_glcm(process_img)
        feature_7.append(GLCM2.flatten())
        labels.append(category_idx)

#data=np.asarray(feature_5).flatten()
#data2=np.asarray(feature_1)
#feature_5=np.asarray(feature_5,dtype="float32")
data=np.asarray(feature_7)
#data = np.column_stack((feature_1,feature_2,feature_3,feature_4,feature_5))
labels = np.asarray(labels)


c:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
c:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
c:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in version 1.0. Use ``skimage.feature.graycomatrix`` instead.
  removed_version='1.0')
c:\Users\DELL\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\feature\__init__.py:35: skimage_deprecation: Function ``greycomatrix`` is deprecated and will be removed in 

In [ ]:
# train / test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [ ]:
# train classifier
classifier = SVC()
#classifier.fit(x_train,y_train)
parameters = [{'gamma': [0.01, 0.001, 0.0001], 'C': [1, 10, 100, 1000]}]

grid_search = GridSearchCV(classifier, parameters)

grid_search.fit(x_train, y_train)

GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001]}])

In [ ]:
# test performance
best_estimator = grid_search.best_estimator_

y_prediction = best_estimator.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

#pickle.dump(best_estimator, open('./model.p', 'wb'))

10.606060606060606% of samples were correctly classified


In [ ]:
'''
***FULL DATASET Egp+SAR***
#SIFT alone = 32%
#HOG alone = 77%
#HOG+H+S+HS= 86%
#H+S+HS+HOG+SIFT= 35 or 41%
#H+HS+S+HOG+LBP=31%
#HOG+H+S+HS(with process_image)=80.2%

*** 33 imgs in dataset***
H,S,HS,HOG (with pre-processing)=77.2%
H,S,HS,HOG (NO pre-processing)=74.2%

HS+HOG (with pre-processing)= 80.3%
HS+HOG (No processing)=78.78%
BOW (No pre)=33.3%
BOW (pre)=37.88%
H (No Pre)= 54.54%
H (Pre)= 56.06%
S (No Pre)= 27.27%
S (Pre)= 34.85%
HS (No Pre)= 56.06%
HS (Pre)= 53.03%
HOG (No Pre)= 77.27%
HOG (Pre)= 80.30%
GLCM (No pre, Co pilot)= 10.606%
GLCM (pre)= 
'''

'\n***DATASET Egp+SAR***\n#SIFT alone = 32%\n#HOG alone = 77%\n#HOG+H+S+HS= 86%\n#H+S+HS+HOG+SIFT= 35 or 41%\n#H+HS+S+HOG+LBP=31%\n#HOG+H+S+HS(with process_image)=80.2%\n\n*** 33 imgs in dataset***\nH,S,HS,HOG (with pre-processing)=77.2%\nHS+HOG (with pre-processing)= 80.3%\n'